In [ ]:
from pdf2image import convert_from_path
import os
import pandas as pd
import io
import time
import re
import json

## Step one: Extract table images from PDFs

In [ ]:
# list all files in original pdf folder
pdf_filepaths_in_folder = os.listdir('original_pdfs')

In [ ]:
print(pdf_filepaths_in_folder)

In [ ]:
# ignore .DS_Store generated by working on mac
pdf_filepaths_in_folder = [x for x in pdf_filepaths_in_folder if x != '.DS_Store']

In [ ]:
for idx, filename in enumerate(pdf_filepaths_in_folder[0:]):
    print(idx)
    print(filename)

    # note this will work for now, but will cause issues for 1978_1979_1980_1981
    year = re.search(r'(\d{4})', filename).group(1)

    print("Converting each page of this PDF to images.")
    images = convert_from_path("original_pdfs/" + filename, grayscale = True, size = (None, 1268))
    print("Done. Saving images to folder.")
    for i in range(len(images)):
        images[i].save('images/file' + str(year) + '_page'+ str(i) +'.jpg', 'JPEG')
    

## Step two: Read ocr manifest

In [ ]:
ocr_manifest = pd.read_csv("ocr_manifest.csv")
ocr_manifest

## Step three: Move photos into import/export subfolders

In [ ]:
for index, row in ocr_manifest.iterrows():
    # for completed rows, go ahead and move stuff around!
    if(pd.isna(row.import_start_n) == False):
        current_year_file = int(row.year)
        import_start_page = int(row.import_start_n)
        import_end_page = int(row.import_end_n)
        export_start_page = int(row.export_start_n)
        export_end_page = int(row.export_end_n)
        # iterate over all pages in this range (need to +1 range because it's exclusive of last number)
        # and move to the correct folder
        for i in range(import_start_page, import_end_page + 1):
           # move import images first
            current_image_path = "file" + str(current_year_file) + "_page" + str(i) + ".jpg"
            # print(current_image_path)
            os.rename("images/" + current_image_path, "images_import_only/" + current_image_path)
        print("Moved all import images from year " + str(current_year_file) + " successfully!")


        # now the same, but for exports
        for i in range(export_start_page, export_end_page + 1):
           # move import images first
            current_image_path = "file" + str(current_year_file) + "_page" + str(i) + ".jpg"
            # print(current_image_path)
            os.rename("images/" + current_image_path, "images_export_only/" + current_image_path)

        
        print("Moved all import images from year " + str(current_year_file) + " successfully!")
        